# Get manuscript 'timeline'

## Description
Given an OpenReview invitation, list the events (submissions, revisions, reviews, comments, etc.) associated with each submission, arranged chronologically.

## Notes
* I am not sure what the difference is between `cdate`, `tcdate` and other timestamps. I will check with OpenReview folks to confirm that I am using the correct field.
* The list of events does not include comment edits, including score changes. I do not think these are available, but I can double check with OpenReview folks on this too.
* I haven't done any of the work to print this into a nice csv or anything, but I think it should be straightforward. You can use `event.stringify(pretty=False)` if you would prefer raw timestamps instead of human readable.
* This code was written with the ICLR 2019 invitation. If you are using a different ICLR invitation, the conditions to identify a metareview might be different.

In [1]:
import collections
from datetime import datetime
import openreview
import os

# Change these values according to your needs
INVITATION = 'ICLR.cc/2019/Conference/-/Blind_Submission'
LIMIT = 1 # Number of papers to build timelines for

# A client is required for any OpenReview API actions
guest_client = openreview.Client(baseurl='https://api.openreview.net')

In [2]:
class EventType(object):
    SUBMISSION = "Submission"
    REVISION = "Revision"
    REVIEW = "Review"
    COMMENT = "Comment"
    METAREVIEW = "Metareview"
    
GENERIC_INITIATOR_AUTHORS = "Authors"

def get_initiator(note):
    return note.signatures[0].split("/")[-1]

def clean_timestamp(timestamp):
    return datetime.fromtimestamp(
        timestamp/1000).strftime('%Y-%m-%dT%H:%M:%S') # Something human-readable for the file name

class Event(object):
    def __init__(self, forum, event_type, initiator, identifier, timestamp, reply_to):
        self.forum = forum
        self.event_type = event_type
        self.initiator = initiator
        self.identifier = identifier
        self.timestamp = timestamp
        self.reply_to = reply_to
    
    def stringify(self, pretty=True):
        if pretty:
            timestamp = clean_timestamp(self.timestamp)
        else:
            timestamp = self.timestamp
        return "\t".join(str(i) for i in [
            timestamp, self.forum, self.event_type,
            self.initiator, self.identifier, self.reply_to])

def get_events_from_forum(forum_note):
    events = []
    for revision in guest_client.get_references(
            referent=forum_note.id, original=True):
        events.append(Event(
            forum_note.id, EventType.REVISION, GENERIC_INITIATOR_AUTHORS,
            revision.id, revision.tcdate, revision.replyto))
  
    this_forum_notes = guest_client.get_notes(forum=forum_note.id)
    for note in this_forum_notes:
        initiator = get_initiator(note)
        
        if 'TL;DR' in note.content: # Find a better test for this!
            event_type = EventType.SUBMISSION
        elif 'review' in note.content:
            event_type = EventType.REVIEW
        elif 'metareview' in note.content:
            event_type = EventType.METAREVIEW
        else:
            event_type = EventType.COMMENT
        events.append(Event(
            forum_note.id, event_type, initiator,
            note.id, note.tcdate, note.replyto))
    return list(sorted(events, key=lambda x:x.timestamp))

In [3]:
for i, forum_note in enumerate(openreview.tools.iterget_notes(
        guest_client, invitation=INVITATION)):
    events = get_events_from_forum(forum_note)
    title = forum_note.content["title"]
    print(f"# {len(events)} events in {title}\n")
    for event in events:
        print(event.stringify())
    print("\n")
    
    if i + 1 == LIMIT:
        break

# 16 events in Deep Decoder: Concise Image Representations from Untrained Non-convolutional Networks

2018-09-27T17:55:01	rylV-2C9KQ	Revision	Authors	BkC3bA5YX	None
2018-09-27T18:38:51	rylV-2C9KQ	Submission	Conference	rylV-2C9KQ	None
2018-10-24T12:29:36	rylV-2C9KQ	Review	AnonReviewer1	BJeYeRM0jm	rylV-2C9KQ
2018-11-02T11:30:36	rylV-2C9KQ	Review	AnonReviewer2	BkeNja15nm	rylV-2C9KQ
2018-11-03T01:58:00	rylV-2C9KQ	Review	AnonReviewer3	S1llxt2cnQ	rylV-2C9KQ
2018-11-13T00:58:04	rylV-2C9KQ	Revision	Authors	BJHguJuTQ	None
2018-11-13T00:59:00	rylV-2C9KQ	Comment	Authors	HJea7OJdpX	S1llxt2cnQ
2018-11-13T01:00:02	rylV-2C9KQ	Comment	Authors	Hkg5wOyOaX	BkeNja15nm
2018-11-13T01:02:02	rylV-2C9KQ	Comment	Authors	r1gzyKy_a7	BJeYeRM0jm
2018-11-14T14:38:53	rylV-2C9KQ	Comment	AnonReviewer1	rkxS0tgca7	r1gzyKy_a7
2018-11-17T18:23:30	rylV-2C9KQ	Comment	Authors	HJeslXQCTX	rkxS0tgca7
2018-12-03T12:52:13	rylV-2C9KQ	Comment	AnonReviewer1	H1xSIp171N	HJeslXQCTX
2018-12-12T18:16:14	rylV-2C9KQ	Metareview	Area_Chair1	B